### Gisement syntéthique

**Impact de la teneur de coupure, de la corrélation spatiale sur la localiation des ressources**

Tout au long de la session, nous allons apprendre à choisir une teneur de coupure, à estimer la corrélation spatiale d’un gisement, et surtout à comprendre comment ces paramètres influencent l’estimation des ressources.

Mais pour l’instant… amusez-vous un peu !

Explorez ce modèle 3D interactif et observez comment la localisation des ressources évolue lorsque vous modifiez la teneur de coupure ou le degré de corrélation spatiale.
Un bon moyen de jouer les géologues tout en développant votre intuition !

**Cette image interactive est encore en développement.**

In [10]:
import numpy as np
from scipy.fftpack import fftn, ifftn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, clear_output

# IMPORTANT : Exécutez la ligne suivante dans une cellule Jupyter SÉPARÉE
# et AVANT tout autre code de ce bloc pour activer l'interactivité 3D.
# %matplotlib widget

# --- Fonctions de simulation géostatistique ---

def spherical_covariance_fft_3d(full_fft_shape, ax, ay, az, angle_x, angle_y, angle_z, sill=1.0):
    """
    Calcule le modèle de covariance sphérique 3D sur une grille adaptée à la simulation
    basée sur la Transformée de Fourier Rapide (FFT).
    """
    nx_fft, ny_fft, nz_fft = full_fft_shape

    # Crée des vecteurs de pas de lag spatiaux en utilisant np.fft.fftfreq
    x_lags = np.fft.fftfreq(nx_fft, d=1) * nx_fft
    y_lags = np.fft.fftfreq(ny_fft, d=1) * ny_fft
    z_lags = np.fft.fftfreq(nz_fft, d=1) * nz_fft
    
    # Crée une grille 3D (meshgrid) pour les lags.
    X_grid, Y_grid, Z_grid = np.meshgrid(x_lags, y_lags, z_lags, indexing='ij')

    # Convertit les angles en radians
    rx, ry, rz = np.radians([angle_x, angle_y, angle_z])

    # Matrices de rotation individuelles pour les axes Z, Y, X (ordre d'Euler conventionnel)
    R_x = np.array([[1, 0, 0],
                    [0, np.cos(rx), -np.sin(rx)],
                    [0, np.sin(rx), np.cos(rx)]])
    R_y = np.array([[np.cos(ry), 0, np.sin(ry)],
                    [0, 1, 0],
                    [-np.sin(ry), 0, np.cos(ry)]])
    R_z = np.array([[np.cos(rz), -np.sin(rz), 0],
                    [np.sin(rz), np.cos(rz), 0],
                    [0, 0, 1]])
    
    # Combinaison des rotations (ordre ZYX standard pour une rotation intrinsèque)
    # C'est souvent l'ordre qui correspond à des rotations intuitives sur un objet
    R_combined = R_z @ R_y @ R_x
    
    # Empile les coordonnées originales dans un tableau de points (N, 3)
    coords_original_flat = np.stack([X_grid.flatten(), Y_grid.flatten(), Z_grid.flatten()], axis=-1)
    
    # Applique la rotation aux coordonnées
    # (N, 3) @ (3, 3).T = (N, 3) @ (3, 3) = (N, 3)
    coords_rotated_flat = coords_original_flat @ R_combined.T # Utilisez R_combined.T pour la transformation de points
    
    # Remodèle les coordonnées rotées à la forme de la grille 3D
    X_rotated = coords_rotated_flat[:, 0].reshape(full_fft_shape)
    Y_rotated = coords_rotated_flat[:, 1].reshape(full_fft_shape)
    Z_rotated = coords_rotated_flat[:, 2].reshape(full_fft_shape)

    # Applique la mise à l'échelle aux coordonnées déjà rotées
    # C'est ici que l'anisotropie est appliquée après la rotation
    X_scaled_rotated = X_rotated / ax
    Y_scaled_rotated = Y_rotated / ay
    Z_scaled_rotated = Z_rotated / az

    # Calcule la distance euclidienne (h) dans l'espace anisotrope et roté.
    h = np.sqrt(X_scaled_rotated**2 + Y_scaled_rotated**2 + Z_scaled_rotated**2)
    
    # Formule du modèle sphérique
    cov = sill * (1 - 1.5 * h + 0.5 * h**3)
    cov[h >= 1] = 0

    return cov

def fftma_3d(shape, ax, ay, az, angle_x, angle_y, angle_z, sill=1.0, seed=0):
    """
    Réalise une simulation géostatistique 3D en utilisant la méthode FFT-Moving Average.
    """
    np.random.seed(seed)
    fft_shape = tuple(2 * s for s in shape)
    
    cov = spherical_covariance_fft_3d(fft_shape, ax, ay, az, angle_x, angle_y, angle_z, sill)
    cov = np.real(cov) 

    white_noise = np.random.normal(loc=0, scale=1, size=fft_shape)
    
    cov_fft = fftn(cov)
    white_fft = fftn(white_noise)
    
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    
    field = np.real(ifftn(z_fft))
    
    slices = tuple(slice(0, s) for s in shape)
    return field[slices]

def gaussian_to_lognormal(field, mean_lognormal, variance_lognormal):
    """
    Transforme un champ Gaussien (normal) standard en un champ lognormal
    avec une moyenne et une variance spécifiées.
    """
    if mean_lognormal <= 0:
        print("Avertissement : La moyenne lognormale doit être positive. Retourne un tableau de zéros.")
        return np.zeros_like(field)

    sigma_g_squared = np.log(variance_lognormal / (mean_lognormal**2) + 1)
    
    if sigma_g_squared < 0:
        print(f"Avertissement : Le sigma_g_squared calculé ({sigma_g_squared:.4f}) est négatif. Ajuste à zéro.")
        sigma_g_squared = 0
        
    sigma_g = np.sqrt(sigma_g_squared)
    mu_g = np.log(mean_lognormal) - 0.5 * sigma_g_squared
    
    transformed_field = np.exp(field * sigma_g + mu_g)
    
    return transformed_field

# --- Fonction de tracé 3D avec Matplotlib ---

def plot_matplotlib_3d(field, cutoff):
    """
    Génère un nuage de points 3D des voxels dont la valeur est supérieure ou égale à un seuil donné,
    en utilisant Matplotlib. Inclut le pourcentage de points au-dessus du seuil et une échelle de couleur fixe.
    """
    nx, ny, nz = field.shape
    
    # Calcul du pourcentage de points au-dessus du seuil
    total_voxels = field.size
    mask = field >= cutoff
    voxels_above_cutoff = np.sum(mask)
    
    percentage_above_cutoff = (voxels_above_cutoff / total_voxels) * 100 if total_voxels > 0 else 0.0

    coords = np.argwhere(mask)

    if len(coords) == 0:
        print(f"Aucun voxel au-dessus du seuil de coupure ({cutoff:.2f} ppm). Pourcentage: {percentage_above_cutoff:.2f}%")
        return None 

    x, y, z = coords.T
    values = field[mask]

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Crée un nuage de points 3D avec échelle de couleur fixe (0 à max du champ)
    scatter = ax.scatter(x, y, z, c=values, cmap='viridis', marker='o', s=10, alpha=0.8,
                         vmin=0, vmax=field.max()) # Fixe la légende entre 0 et la valeur max du champ

    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')

    # Titre incluant le pourcentage
    ax.set_title(f'Simulation 3D (Teneur > {cutoff:.2f} ppm)\nPourcentage de voxels: {percentage_above_cutoff:.2f}%')

    cbar = fig.colorbar(scatter, ax=ax, pad=0.1)
    cbar.set_label('Teneur (ppm)')

    ax.set_xlim(0, nx)
    ax.set_ylim(0, ny)
    ax.set_zlim(0, nz)

    return fig

# --- Widgets et logique d'interaction ---

# Widgets pour l'interaction utilisateur
mean_w = widgets.FloatText(value=1.0, description='Moyenne Lognormale (ppm)')
variance_w = widgets.FloatText(value=2.0, description='Variance Lognormale (ppm²)')
cutoff_w = widgets.FloatText(value=2.0, description='Teneur coupure (ppm)')
ax_w = widgets.FloatText(value=20.0, description='Portée Ax (m)')
ay_w = widgets.FloatText(value=20.0, description='Portée Ay (m)')
az_w = widgets.FloatText(value=20.0, description='Portée Az (m)')
angle_x_w = widgets.IntSlider(value=0, min=-90, max=90, step=5, description='Angle X (°)')
angle_y_w = widgets.IntSlider(value=0, min=-90, max=90, step=5, description='Angle Y (°)')
angle_z_w = widgets.IntSlider(value=0, min=-90, max=90, step=5, description='Angle Z (°)')
seed_w = widgets.IntText(value=42, description='Graine aléatoire')

button = widgets.Button(description='Exécuter la simulation', button_style='primary', icon='play')

output = widgets.Output()

# Gestionnaire d'événements pour le clic sur le bouton
def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        print("Génération de la simulation... Veuillez patienter. Cela peut prendre quelques secondes pour de grandes tailles.")
        
        shape = (100, 100, 50) # Taille de la simulation (voxels)

        try:
            gaussian_field = fftma_3d(shape, ax_w.value, ay_w.value, az_w.value,
                                      angle_x_w.value, angle_y_w.value, angle_z_w.value,
                                      sill=1.0, 
                                      seed=seed_w.value)
            
            lognormal_field = gaussian_to_lognormal(gaussian_field, mean_w.value, variance_w.value)
            
            matplotlib_fig = plot_matplotlib_3d(lognormal_field, cutoff_w.value)
            
            if matplotlib_fig:
                display(matplotlib_fig)
        
        except Exception as e:
            print(f"Une erreur est survenue lors de la simulation ou du tracé : {e}")
            print("Veuillez vérifier les paramètres d'entrée ou la taille de la simulation.")

button.on_click(on_button_clicked)

# Agencement et affichage des widgets
ui = widgets.VBox([
    widgets.HBox([mean_w, variance_w, cutoff_w]),
    widgets.HBox([ax_w, ay_w, az_w]),
    widgets.HBox([angle_x_w, angle_y_w, angle_z_w]),
    widgets.HBox([seed_w]),
    button,
    output
])

display(ui)